## Process Sentinel-3 LST data
- Subset data to region of interest
- Reproject
- Remove cloud
- Only save LST as GeoTIFF where Minimum number of valid pixels(not nan) > pixel_threshold

In [ ]:
import os
from esa_snappy import ProductIO, HashMap, GPF, jpy
import esa_snappy
import numpy as np
import logging
import csv
import gc
import tempfile
import shutil

# Configuration
input_directory = r"F:\GoogleDrive\sentinel\LST zips"   # Input directory containing Sentinel-3 ZIP files
output_subdir = "processed" # Output subdirectory for processed files
os.makedirs(os.path.join(input_directory, output_subdir), exist_ok=True)

# Define the WKT for the geographic region of interest(for subset)
wkt = "POLYGON ((-74.257 40.495, -74.257 40.916, -73.699 40.916, -73.699 40.495, -74.257 40.495))"
reference_product = ProductIO.readProduct(r'F:\GoogleDrive\sentinel\LST zips\reference.tif')# Read reference product for reprojection (if any)
rprj_coord = "EPSG:32618"   # Target projection
pixel_threshold = 100  # Minimum number of valid pixels(not nan) required to save the product

# Configure logging
logging.basicConfig(
    filename=os.path.join(input_directory, output_subdir, 'error_log.log'),
    level=logging.ERROR,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Get all ZIP files in the directory
zip_files = [f for f in os.listdir(input_directory) if f.endswith('.zip')]

def count_non_nan_pixels(band):
    """Count non-NaN pixels in a given band."""
    width = band.getRasterWidth()
    height = band.getRasterHeight()
    pixels = np.zeros((width * height,), dtype=np.float32)
    band.readPixels(0, 0, width, height, pixels)
    return np.count_nonzero(~np.isnan(pixels))

error_list = []

for zipfile in zip_files:
    try:
        # Read input product
        product = ProductIO.readProduct(f'{input_directory}\{zipfile}')
        
        # Subset by geographic region
        WKTReader = jpy.get_type('org.locationtech.jts.io.WKTReader')
        geom = WKTReader().read(wkt)

        band_names = ['LST', 'bayes_in']
        subset_params = HashMap()
        subset_params.put('copyMetadata', True)
        subset_params.put('sourceBands', ','.join(band_names))
        subset_params.put('geoRegion', geom)
        subset_params.put("cacheSize", 0)  # Disable SNAP cache
        subset_params.put("writeTemporary", False)  # Disable temporary files
        subset = GPF.createProduct('Subset', subset_params, product)
        
        # Reproject to match reference product
        reproject_params = HashMap()
        sources = HashMap()
        sources.put('source', subset)
        if reference_product is not None:
            sources.put('collocateWith', reference_product)
        else:
            reproject_params.put("crs", rprj_coord)
        reproject_params.put("orthorectify", False)
        reproject_params.put("noDataValue", float('nan'))
        reproject_params.put("includeTiePointGrids", False)# https://forum.step.esa.int/t/slight-shift-when-subset-reproject-some-s3-wfr-products-using-snappy/35610
        reproject = GPF.createProduct('Reproject', reproject_params, sources)
        
        # Apply cloud mask using Bayes probability
        mask_params = HashMap()
        BandDescriptor = jpy.get_type('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor')
        targetBand = BandDescriptor()
        targetBand.name = 'Mask'
        targetBand.type = 'float32'
        targetBand.expression = 'if bayes_in_single_moderate == 0 then LST else NaN'
        targetBands = jpy.array('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor', 1)
        targetBands[0] = targetBand
        mask_params.put('targetBands', targetBands)
        masked_product = GPF.createProduct('BandMaths', mask_params, reproject)
        
        # Save only if sufficient valid pixels exist
        mask_band = masked_product.getBand('Mask')
        valid_pixel_count = count_non_nan_pixels(mask_band)
        
        if valid_pixel_count > pixel_threshold:
            output_path = os.path.join(input_directory, output_subdir, f'{zipfile.split(".")[0]}.tif')
            esa_snappy.GPF.writeProduct(
                masked_product, 
                esa_snappy.File(output_path), 
                'GeoTIFF', 
                False,  # incremental
                esa_snappy.ProgressMonitor.NULL
            )
            print(f'Product successfully saved to: {output_path}')
        else:
            print(f"Skipped {zipfile}: Only {valid_pixel_count} valid pixels (threshold: {pixel_threshold})")
            
    except Exception as e:
        error_list.append(zipfile)
        logging.error(f"Error processing {zipfile}: {str(e)}")
        
    finally:
        # Clean up resources     
        temp_dir = tempfile.gettempdir()
        shutil.rmtree(temp_dir, ignore_errors=True)
        gc.collect()
    
# Write error report
with open(os.path.join(input_directory, output_subdir, 'error.csv'), 'w', newline='') as f:
    csv.writer(f).writerows([[item] for item in error_list])

## This script processes Sentinel-3 TIFF filenames to extract metadata including:
 - Sensor ID
 - Acquisition date/time
 - Orbit/tile numbers
 - Data dissemination level
 - Product criticality
 Results are saved to a CSV file for analysis.

In [ ]:
import os
import pandas as pd

# Configuration
input_dir = os.path.join(input_directory, output_subdir)

# Get all TIFF files in output directory
tif_files = [f for f in os.listdir(input_dir) 
             if f.endswith('.tif')]

# Process each filename to extract metadata
metadata = []
for filename in tif_files:
    # Extract components from filename using string positions
    sensor = filename[:3]  # Satellite identifier (e.g., S3B)
    date = filename[16:24]  # Acquisition date (YYYYMMDD)
    
    # Format time from T151914 to 15:19:14
    time_str = filename[24:31]
    formatted_time = f'{time_str[1:3]}:{time_str[3:5]}:{time_str[5:7]}'
    
    # Extract additional metadata fields
    record = [
        filename,  # Original filename
        sensor,
        date,
        formatted_time,
        time_str[1:3],  # Hour only
        filename[69:72],  # Orbit number
        filename[73:76],  # Tile number
        filename[86],  # Dissemination level
        filename[-10:-4]  # Criticality code
    ]
    metadata.append(record)

# Create and save DataFrame
columns = [
    'filename', 'sensor', 'date', 'time', 'hour',
    'orbit', 'tile', 'dissemination_level', 'criticality'
]
pd.DataFrame(metadata, columns=columns).to_csv(os.path.join(input_dir, 'metadata.csv'))